<a href="https://colab.research.google.com/github/Sharpoint2/Spotify-New-Release-Finder/blob/main/New_Release_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import base64
import time
from urllib.parse import urlencode

# --- Your Spotify API Credentials ---
# Replace these with the credentials from your Spotify Developer Dashboard App
CLIENT_ID = "CLIENT_ID"
CLIENT_SECRET = "CLIENT_SECRET"
# ------------------------------------

# --- Customize Your Search ---
# Add or remove genres to tailor your search for new releases.
GENRE_LIST = [
    "rock", "alt-rock", "indie", "punk", "hardcore",
    "metal", "death-metal", "black-metal", "deathcore",
    "hip-hop", "trap","rap",
    "pop", "synth-pop",
    "electronic", "techno", "house", "ambient",
    "jazz", "soul", "funk", "blues",
    "classical", "folk", "country", "metalcore"
]
# -----------------------------


def get_spotify_token(client_id, client_secret):
    """Gets an access token from the Spotify API."""
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode('utf-8')).decode('utf-8')
    headers = {'Authorization': f'Basic {auth_header}'}
    data = {'grant_type': 'client_credentials'}
    response = requests.post(auth_url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json().get('access_token')
    else:
        print(f"Error getting token: {response.status_code} - {response.text}")
        return None

def search_new_releases_by_genre(token, genres):
    """Searches for new releases across a list of genres, correctly handling pagination."""
    all_new_releases = []
    seen_album_ids = set()

    headers = {'Authorization': f'Bearer {token}'}
    base_search_url = 'https://api.spotify.com/v1/search'

    for genre in genres:
        print(f"\nSearching for new releases in genre: '{genre}'...")

        # --- THE FIX IS IMPLEMENTED HERE ---

        # 1. Start with the base URL and the parameters for the first search.
        next_url = base_search_url
        params = {
            'q': f'genre:"{genre}" tag:new',
            'type': 'album',
            'limit': 50
        }

        # Loop to handle all pages of results for the current genre
        while next_url:
            # 2. Make the request using the URL and any available params.
            # The `requests` library correctly handles if `params` is None.
            response = requests.get(next_url, headers=headers, params=params)

            # 3. CRITICAL: Clear the params after the first request.
            # The 'next_url' from Spotify contains all needed parameters,
            # so we must not send them again.
            if params:
                params = None

            if response.status_code == 200:
                data = response.json()
                albums = data.get('albums', {}).get('items', [])

                found_count = 0
                for album in albums:
                    if album['id'] not in seen_album_ids:
                        all_new_releases.append(album)
                        seen_album_ids.add(album['id'])
                        found_count += 1

                if found_count > 0:
                    print(f"   > Found {found_count} new unique releases.")

                # Get the URL for the next page of results
                next_url = data.get('albums', {}).get('next')
                time.sleep(0.1)
            elif response.status_code == 429:
                retry_after = int(response.headers.get('Retry-After', 1))
                print(f"   > Rate limit hit. Waiting for {retry_after} seconds...")
                time.sleep(retry_after)
            else:
                print(f"   > Error searching for genre '{genre}': {response.status_code} - {response.text}")
                break # Stop searching this genre if an error occurs

    return all_new_releases

def display_releases(releases):
    """Prints the list of releases in a readable format."""
    if not releases:
        print("No releases found or an error occurred.")
        return

    print("\n\n--- 🎸 A More Comprehensive List of New Releases ---")

    # Sort releases by release date, most recent first
    releases.sort(key=lambda x: x['release_date'], reverse=True)

    for i, item in enumerate(releases, 1):
        album_type = item['album_type'].capitalize()
        album_name = item['name']
        release_date = item['release_date']
        artist_names = ', '.join([artist['name'] for artist in item['artists']])
        spotify_url = item['external_urls']['spotify']

        print(f"\n{i}. [{album_type}] {artist_names} - {album_name}")
        print(f"   Released: {release_date}")
        print(f"   Listen here: {spotify_url}")

    print(f"\n--- Total Unique Releases Found: {len(releases)} ---")


if __name__ == "__main__":
    if CLIENT_ID == "YOUR_CLIENT_ID_HERE" or CLIENT_SECRET == "YOUR_CLIENT_SECRET_HERE":
        print("🛑 ERROR: Please replace the placeholder credentials in the script.")
    else:
        access_token = get_spotify_token(CLIENT_ID, CLIENT_SECRET)
        if access_token:
            new_releases = search_new_releases_by_genre(access_token, GENRE_LIST)
            display_releases(new_releases)


Searching for new releases in genre: 'rock'...

Searching for new releases in genre: 'alt-rock'...

Searching for new releases in genre: 'indie'...

Searching for new releases in genre: 'punk'...

Searching for new releases in genre: 'hardcore'...

Searching for new releases in genre: 'metal'...

Searching for new releases in genre: 'death-metal'...

Searching for new releases in genre: 'black-metal'...

Searching for new releases in genre: 'deathcore'...

Searching for new releases in genre: 'hip-hop'...
   > Found 1 new unique releases.

Searching for new releases in genre: 'trap'...

Searching for new releases in genre: 'rap'...

Searching for new releases in genre: 'pop'...

Searching for new releases in genre: 'synth-pop'...

Searching for new releases in genre: 'electronic'...

Searching for new releases in genre: 'techno'...
   > Found 1 new unique releases.

Searching for new releases in genre: 'house'...

Searching for new releases in genre: 'ambient'...

Searching for new re

In [ ]:
import requests
import base64
import time
from datetime import datetime, timedelta, timezone

# --- Your Spotify API Credentials ---
CLIENT_ID = "f7a1820968e54174840f1284ed125c56"
CLIENT_SECRET = "f5f4d35c3aa94200a62b0b9a9b524dd5"
# ------------------------------------

# --- Configuration ---
TARGET_USER_ID = "spotify"
# -----------------------------


def get_spotify_token(client_id, client_secret):
    """Gets an access token from the Spotify API."""
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode('utf-8')).decode('utf-8')
    headers = {'Authorization': f'Basic {auth_header}'}
    data = {'grant_type': 'client_credentials'}
    response = requests.post(auth_url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json().get('access_token')
    else:
        print(f"Error getting token: {response.status_code} - {response.text}")
        return None

def get_all_user_playlists(token, user_id):
    """Fetches all public playlists from a specific user's profile."""
    headers = {'Authorization': f'Bearer {token}'}
    all_playlists = []
    # This is the correct endpoint for getting a user's playlists
    next_url = f'https://api.spotify.com/v1/users/{user_id}/playlists?limit=50'

    print(f"Fetching all public playlists from user '{user_id}'... This will take a moment.")

    while next_url:
        response = requests.get(next_url, headers=headers)
        if response.status_code != 200:
            print(f"   > Error fetching playlist data: {response.status_code} - {response.text}")
            break

        data = response.json()
        all_playlists.extend(data.get('items', []))
        next_url = data.get('next')
        time.sleep(0.1)

    return all_playlists

def display_profile_playlists(playlists):
    """Displays the fetched playlists in a readable format."""
    if not playlists:
        print("Could not find any playlists for this user.")
        return

    print(f"\n--- 🎧 Found {len(playlists)} Playlists on Spotify's Profile ---")

    for i, playlist in enumerate(playlists, 1):
        name = playlist['name']
        playlist_id = playlist['id']
        # Truncate long descriptions for cleaner output
        description = (playlist['description'] or "No description.").replace('\n', ' ').strip()
        if len(description) > 100:
            description = description[:97] + "..."

        print(f"\n{i}. {name}")
        print(f"   ID: {playlist_id}")
        print(f"   Description: {description}")


if __name__ == "__main__":
    access_token = get_spotify_token(CLIENT_ID, CLIENT_SECRET)
    if access_token:
        spotify_playlists = get_all_user_playlists(access_token, TARGET_USER_ID)
        display_profile_playlists(spotify_playlists)

Fetching all public playlists from user 'spotify'... This will take a moment.

--- 🎧 Found 346 Playlists on Spotify's Profile ---

1. Classic Honky Tonk
   ID: 0NfjMqrzcGKVsbYZmhf4Md
   Description: All songs about drinking, cheating, heartaches and everything else going on in a classic honky tonk.

2. Nicholas Sparks | Songs from the Soundtracks
   ID: 1scnlLVq91NGtsA9sh0hfw
   Description: Go behind the scenes of the <a href="http://www.twobytwomusic.com">Two By Two</a> book soundtrack...

3. 1960s Nostalgia
   ID: 2NFOUmp2wyR5CrXtKDkUkB
   Description: Those were the days, this was the music...

4. Breakup Blues
   ID: 1o2bTwofazfzElA5mXGf2t
   Description: From denial and pleading to anger and acceptance, this genre-blending playlist will carry you thr...

5. I Hate My Job.
   ID: 6cdV0hVW2suJaMOxzwE46S
   Description: We've all been there. This playlist can help. Just don't let your boss catch you listening....

6. Instrumental Rock
   ID: 2uhsnHgI4F2eFyvoMHY0GR
   Description: Ro